In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from datetime import datetime, timezone

%reload_ext autoreload
%autoreload 2
from data.raw.retrievers.alpaca_markets_retriever import AlpacaMarketsRetriever
from data.processed.indicators import *
from data.processed.targets import BinaryClassification, MeanReturnSignClassification, Balanced5ClassClassification
from data.processed.normalization import ZScoreNormalizer, ZScoreOverWindowNormalizer, MinMaxNormalizer
from data.processed.data_processing_utils import *
from config.constants import Constants
from modeling.evaluate import evaluate_test_return, evaluate_lgb_classifier, evaluate_lgb_regressor
from data.processed.missing_values_handling import ForwardFillFlatBars, DummyMissingValuesHandler
from data.processed.dataset_creation import DatasetCreator


ImportError: cannot import name 'evaluate_test_return' from 'modeling.evaluate' (c:\Users\ikurnosau\Projects\QuantitativeTrading\intraday-portfolio-management\modeling\evaluate.py)

In [2]:
retriever = AlpacaMarketsRetriever()

retrieval_result = retriever.bars(
    symbol_or_symbols='SE', 
    start=datetime(2024, 5, 14), 
    end=datetime(2025, 5, 14))
df = pd.concat(retrieval_result.values()).reset_index(drop=True)
df

,date,open,high,low,close,volume
0,2024-05-14 08:00:00+00:00,64.3900,64.3900,64.0000,64.0000,2279.0
1,2024-05-14 08:01:00+00:00,64.0000,64.3500,63.9800,64.3500,3579.0
2,2024-05-14 08:02:00+00:00,64.0500,64.0500,64.0500,64.0500,1000.0
3,2024-05-14 08:03:00+00:00,64.2800,64.3000,64.0000,64.3000,2092.0
4,2024-05-14 08:07:00+00:00,63.9400,64.0100,63.9400,64.0100,1704.0
...,...,...,...,...,...,...
108419,2025-05-13 23:07:00+00:00,156.9367,156.9367,156.9367,156.9367,100.0
108420,2025-05-13 23:39:00+00:00,156.8498,156.8498,156.8498,156.8498,1052.0
108421,2025-05-13 23:40:00+00:00,156.8500,156.8500,156.8500,156.8500,268.0
108422,2025-05-13 23:52:00+00:00,156.8400,156.8400,156.8400,156.8400,100.0


In [5]:
filter_by_regular_hours(df, 'date')['close'].pct_change().dropna().apply(lambda r: r > 0).astype(np.float32).mean()

0.47186926

In [6]:
df.columns

Index(['date', 'rtype_x', 'publisher_id_x', 'instrument_id_x', 'open', 'high',
       'low', 'close', 'volume', 'symbol_x', 'Unnamed: 0', 'ts_recv',
       'rtype_y', 'publisher_id_y', 'instrument_id_y', 'action', 'side',
       'depth', 'price', 'size', 'flags', 'ts_in_delta', 'sequence',
       'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_ct_00',
       'ask_ct_00', 'bid_px_01', 'ask_px_01', 'bid_sz_01', 'ask_sz_01',
       'bid_ct_01', 'ask_ct_01', 'bid_px_02', 'ask_px_02', 'bid_sz_02',
       'ask_sz_02', 'bid_ct_02', 'ask_ct_02', 'bid_px_03', 'ask_px_03',
       'bid_sz_03', 'ask_sz_03', 'bid_ct_03', 'ask_ct_03', 'bid_px_04',
       'ask_px_04', 'bid_sz_04', 'ask_sz_04', 'bid_ct_04', 'ask_ct_04',
       'bid_px_05', 'ask_px_05', 'bid_sz_05', 'ask_sz_05', 'bid_ct_05',
       'ask_ct_05', 'bid_px_06', 'ask_px_06', 'bid_sz_06', 'ask_sz_06',
       'bid_ct_06', 'ask_ct_06', 'bid_px_07', 'ask_px_07', 'bid_sz_07',
       'ask_sz_07', 'bid_ct_07', 'ask_ct_07', 'bid_px_08', 'a

In [11]:
# features={
#     "volume": lambda data: data['volume'],
#     "return": lambda data: data['close'].pct_change(),
#     "return_H": lambda data: data['high'] / data['close'].shift(1) - 1,
#     "return_L": lambda data: data['low'] / data['close'].shift(1) - 1,
#     "OBV": OBV(base_feature='return'),
#     "RSI6": RSI(6, base_feature='return'),
#     "RSI12": RSI(12, base_feature='return'),
#     "EMA3": EMA(3, base_feature='close'),
#     "EMA6": EMA(6, base_feature='return'),
#     "EMA12": EMA(12, base_feature='return'),
#     "ATR14": ATR(14, high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "MFI": MFI(14, high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "ADX14": ADX(14, high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "ADX20": ADX(20, high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "MOM1": MOM(1, base_feature='return'),
#     "MOM3": MOM(3, base_feature='return'),
#     "CCI12": CCI(12, high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "CCI20": CCI(20, high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "ROCR12": ROCR(12, base_feature='return'),
#     "MACD": MACD(base_feature='return'),
#     "WILLR": WILLR(10, high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "TRIX": TRIX(20, base_feature='return'),
#     "BB_LOW": BollingerBand(BollingerBand.BBType.LOWER, base_feature='return'),
#     "BB_UP": BollingerBand(BollingerBand.BBType.UPPER, base_feature='return'),
#     "EMA_26": EMA(26, base_feature="return"),
#     "VWAP": VWAP(high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "ATR_28": ATR(28, high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "FRL_0": FRL(FRL.FIB_RATIOS[0], high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "FRL_1": FRL(FRL.FIB_RATIOS[1], high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "FRL_2": FRL(FRL.FIB_RATIOS[2], high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "FRL_3": FRL(FRL.FIB_RATIOS[3], high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "FRL_4": FRL(FRL.FIB_RATIOS[4], high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "RSI_28": RSI(24, base_feature='return'),
#     "Oscillator_K": Oscillator(Oscillator.LineType.K, high_feature='return_H', low_feature='return_L', close_feature='return'),
#     "Oscillator_D": Oscillator(Oscillator.LineType.D, high_feature='return_H', low_feature='return_L', close_feature='return'),
# }
features={
    "open": lambda data: data['open'],
    "high": lambda data: data['high'],
    "low": lambda data: data['low'],
    "close": lambda data: data['close'],
    "volume": lambda data: data['volume'],
    "return": lambda data: data['close'].pct_change(),
    "OBV": OBV(),
    "RSI6": RSI(6),
    "RSI12": RSI(12),
    "EMA3": EMA(3),
    "EMA6": EMA(6),
    "EMA12": EMA(12),
    "ATR14": ATR(14),
    "MFI": MFI(14),
    "ADX14": ADX(14),
    "ADX20": ADX(20),
    "MOM1": MOM(1),
    "MOM3": MOM(3),
    "CCI12": CCI(12),
    "CCI20": CCI(20),
    "ROCR12": ROCR(12),
    "MACD": MACD(),
    "WILLR": WILLR(10),
    "TRIX": TRIX(20),
    "BB_LOW": BollingerBand(BollingerBand.BBType.LOWER),
    "BB_UP": BollingerBand(BollingerBand.BBType.UPPER),
    "EMA_26": EMA(26, base_feature="close"),
    "VWAP": VWAP(high_feature='high', low_feature='low', close_feature='close'),
    "ATR_28": ATR(28, high_feature='high', low_feature='low', close_feature='close'),
    "FRL_0": FRL(FRL.FIB_RATIOS[0], high_feature='high', low_feature='low', close_feature='close'),
    "FRL_1": FRL(FRL.FIB_RATIOS[1], high_feature='high', low_feature='low', close_feature='close'),
    "FRL_2": FRL(FRL.FIB_RATIOS[2], high_feature='high', low_feature='low', close_feature='close'),
    "FRL_3": FRL(FRL.FIB_RATIOS[3], high_feature='high', low_feature='low', close_feature='close'),
    "FRL_4": FRL(FRL.FIB_RATIOS[4], high_feature='high', low_feature='low', close_feature='close'),
    "RSI_28": RSI(24),
    "Oscillator_K": Oscillator(Oscillator.LineType.K),
    "Oscillator_D": Oscillator(Oscillator.LineType.D),
}

train_last_date = datetime(2025, 4, 13, tzinfo=timezone.utc)

target = Balanced5ClassClassification(horizon=1, base_feature='close')
# target = BinaryClassification(base_feature='close')
if hasattr(target, 'fit'):
    print('Fitting target')
    target.fit(df[df['date'] <= train_last_date])

Fitting target


In [7]:
additional_features = ['bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_ct_00',
       'ask_ct_00', 'bid_px_01', 'ask_px_01', 'bid_sz_01', 'ask_sz_01',
       'bid_ct_01', 'ask_ct_01', 'bid_px_02', 'ask_px_02', 'bid_sz_02',
       'ask_sz_02', 'bid_ct_02', 'ask_ct_02', 'bid_px_03', 'ask_px_03',
       'bid_sz_03', 'ask_sz_03', 'bid_ct_03', 'ask_ct_03', 'bid_px_04',
       'ask_px_04', 'bid_sz_04', 'ask_sz_04', 'bid_ct_04', 'ask_ct_04']
for feature in additional_features:
    features[feature] = lambda data: data[feature]

In [12]:
dataset_creator = DatasetCreator(
    features=features,
    target=Balanced5ClassClassification(horizon=1, base_feature='close'),
    normalizer=MinMaxNormalizer(fit_feature=None),
    missing_values_handler=ForwardFillFlatBars(),
    train_set_last_date=datetime(2025, 4, 14, tzinfo=timezone.utc), 
    in_seq_len=1,
    multi_asset_prediction=False,
)

X_train, y_train, next_return_train, X_test, y_test, next_return_test = dataset_creator.create_dataset_numpy(retrieval_result)
X_train.shape, y_train.shape, next_return_train.shape, X_test.shape, y_test.shape, next_return_test.shape

((80379, 5), (80379,), (80379,), (7370, 5), (7370,), (7370,))

In [13]:
y_train.mean(), y_test.mean()

(0.4989145, 0.4968453)

In [14]:
evaluate_lgb_regressor(X_train, y_train, X_test, y_test, next_return_test)

c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000292 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 80379, number of used features: 5
[LightGBM] [Info] Start training from score 0.498915


c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Train rmse: 0.3479012717882749, Test rmse: 0.3689338502226185, Baseline rmse: 0.36625170707702637, Expected return: 2.8248153634184874e-05


c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [33]:
abs(next_return_test).mean()

0.0007176705

In [11]:
for feature_name, feature_func in features.items():
    df[feature_name] = feature_func(df)

df['target'] = target(df)
df.head()

,date,open,high,low,close,volume,vwap,return,OBV,RSI6,...,ATR_28,FRL_0,FRL_1,FRL_2,FRL_3,FRL_4,RSI_28,Oscillator_K,Oscillator_D,target
0,2024-06-03 08:00:00+00:00,192.45,193.14,192.45,192.77,3376.0,192.924133,NaN,0.0,0.000000,...,0.690000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.00
1,2024-06-03 08:01:00+00:00,192.69,192.69,192.69,192.69,1028.0,192.690000,-0.000415,-1028.0,0.000000,...,0.647931,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.50
2,2024-06-03 08:02:00+00:00,192.68,192.70,192.68,192.70,6174.0,192.695860,0.000052,5146.0,11.111107,...,0.604625,NaN,NaN,NaN,NaN,NaN,11.111107,NaN,NaN,0.75
3,2024-06-03 08:03:00+00:00,192.73,192.78,192.73,192.74,2982.0,192.758901,0.000208,8128.0,38.461527,...,0.568444,NaN,NaN,NaN,NaN,NaN,38.461527,NaN,NaN,1.00
4,2024-06-03 08:04:00+00:00,192.77,193.00,192.77,193.00,13024.0,192.836182,0.001349,21152.0,79.487169,...,0.547172,NaN,NaN,NaN,NaN,NaN,79.487169,NaN,NaN,1.00


In [12]:
df[['return', 'target']].head(10)

,return,target
0,NaN,0.00
1,-0.000415,0.50
2,0.000052,0.75
3,0.000208,1.00
4,0.001349,1.00
5,0.000777,0.00
6,-0.000362,0.50
7,0.000052,0.50
8,0.000052,0.00
9,-0.001036,0.50


In [13]:
# df[list(features.keys())] = ZScoreOverWindowNormalizer(700)(df[list(features.keys())])
# normalizer = MinMaxNormalizer(fit_feature='close').fit(df[df['date'] <= train_last_date][list(features.keys())])
normalizer = MinMaxNormalizer(fit_feature=None).fit(df[df['date'] <= train_last_date][list(features.keys())])
# normalizer = ZScoreNormalizer()#.fit(df[df['date'] <= train_last_date][list(features.keys())])
# normalizer = ZScoreOverWindowNormalizer(600, fit_feature='close')
df[list(features.keys())] = normalizer(df[list(features.keys())])
df

,date,open,high,low,close,volume,vwap,return,OBV,RSI6,...,ATR_28,FRL_0,FRL_1,FRL_2,FRL_3,FRL_4,RSI_28,Oscillator_K,Oscillator_D,target
0,2024-06-03 08:00:00+00:00,0.264019,0.266754,0.266601,0.267702,0.000026,192.924133,NaN,0.375487,0.000000,...,0.313218,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.00
1,2024-06-03 08:01:00+00:00,0.266638,0.261827,0.269218,0.266829,0.000007,192.690000,0.511560,0.375484,0.000000,...,0.293898,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.50
2,2024-06-03 08:02:00+00:00,0.266528,0.261936,0.269109,0.266938,0.000048,192.695860,0.515015,0.375500,0.111111,...,0.274009,NaN,NaN,NaN,NaN,NaN,0.112649,NaN,NaN,0.75
3,2024-06-03 08:03:00+00:00,0.267074,0.262812,0.269654,0.267374,0.000023,192.758901,0.516167,0.375508,0.384615,...,0.257393,NaN,NaN,NaN,NaN,NaN,0.389938,NaN,NaN,1.00
4,2024-06-03 08:04:00+00:00,0.267510,0.265221,0.270091,0.270209,0.000102,192.836182,0.524612,0.375542,0.794872,...,0.247624,NaN,NaN,NaN,NaN,NaN,0.805872,NaN,NaN,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188140,2025-05-30 23:55:00+00:00,0.348575,0.344284,0.351111,0.348935,0.000003,200.213356,0.515370,0.053084,0.200000,...,0.013209,0.347727,0.346022,0.344679,0.343684,0.342452,0.520351,0.250,0.125000,0.50
188141,2025-05-30 23:56:00+00:00,0.348789,0.344284,0.351325,0.348935,0.000003,200.220000,0.514631,0.053084,0.498752,...,0.012045,0.347727,0.346022,0.344679,0.343684,0.342452,0.520351,0.250,0.166667,0.50
188142,2025-05-30 23:57:00+00:00,0.348898,0.344393,0.351434,0.349044,0.000004,200.230000,0.515000,0.053086,0.599999,...,0.011278,0.347727,0.346022,0.344679,0.343684,0.342452,0.563654,0.375,0.291667,0.75
188143,2025-05-30 23:58:00+00:00,0.348789,0.344932,0.351325,0.349580,0.000015,200.255554,0.516449,0.053091,0.803245,...,0.012122,0.347727,0.346022,0.344679,0.343684,0.342452,0.592645,0.990,0.538333,0.50


In [14]:
df = filter_by_regular_hours(df, 'date')
df

,date,open,high,low,close,volume,vwap,return,OBV,RSI6,...,ATR_28,FRL_0,FRL_1,FRL_2,FRL_3,FRL_4,RSI_28,Oscillator_K,Oscillator_D,target
0,2024-06-03 13:30:00+00:00,0.270129,0.268944,0.270418,0.273699,0.013355,193.032658,0.526514,0.379853,0.681632,...,0.062844,0.264654,0.264162,0.263805,0.263839,0.263880,0.658510,0.976744,0.929108,0.00
1,2024-06-03 13:31:00+00:00,0.273293,0.269492,0.268019,0.267811,0.003575,193.037786,0.493964,0.378679,0.524476,...,0.083912,0.264784,0.264373,0.264081,0.264180,0.264302,0.502294,0.329670,0.711513,0.25
2,2024-06-03 13:32:00+00:00,0.267565,0.264017,0.267365,0.267592,0.002205,192.697289,0.513863,0.377955,0.559701,...,0.089591,0.264784,0.264373,0.264081,0.264180,0.264302,0.498203,0.307692,0.538036,0.75
3,2024-06-03 13:33:00+00:00,0.267347,0.265002,0.268673,0.267811,0.001637,192.801844,0.515399,0.378493,0.385417,...,0.093928,0.264784,0.264373,0.264081,0.264180,0.264302,0.493019,0.329670,0.322344,1.00
4,2024-06-03 13:34:00+00:00,0.268056,0.267740,0.269872,0.271627,0.002273,193.063506,0.528064,0.379239,0.535433,...,0.102400,0.264784,0.264373,0.264081,0.264180,0.264302,0.563066,0.714286,0.450549,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96858,2025-05-30 19:56:00+00:00,0.364390,0.362352,0.366263,0.366654,0.004605,201.763287,0.521235,0.007308,0.845570,...,0.142870,0.350011,0.351222,0.352237,0.353596,0.355277,0.756324,0.988262,0.868930,0.75
96859,2025-05-30 19:57:00+00:00,0.366463,0.363022,0.368771,0.367362,0.005688,201.875391,0.517014,0.009176,0.826211,...,0.136286,0.350675,0.351888,0.352906,0.354266,0.355949,0.758607,0.990324,0.980362,0.00
96860,2025-05-30 19:58:00+00:00,0.367172,0.363338,0.364737,0.362565,0.008021,201.745008,0.498507,0.006542,0.677570,...,0.142787,0.350750,0.352010,0.353064,0.354463,0.356192,0.677161,0.779269,0.919285,0.00
96861,2025-05-30 19:59:00+00:00,0.362317,0.357862,0.355904,0.353623,0.019922,201.065191,0.484517,0.000000,0.535055,...,0.158975,0.350750,0.352010,0.353064,0.354463,0.356192,0.557535,0.409883,0.726492,1.00


In [15]:
df = df.dropna()
df

,date,open,high,low,close,volume,vwap,return,OBV,RSI6,...,ATR_28,FRL_0,FRL_1,FRL_2,FRL_3,FRL_4,RSI_28,Oscillator_K,Oscillator_D,target
0,2024-06-03 13:30:00+00:00,0.270129,0.268944,0.270418,0.273699,0.013355,193.032658,0.526514,0.379853,0.681632,...,0.062844,0.264654,0.264162,0.263805,0.263839,0.263880,0.658510,0.976744,0.929108,0.00
1,2024-06-03 13:31:00+00:00,0.273293,0.269492,0.268019,0.267811,0.003575,193.037786,0.493964,0.378679,0.524476,...,0.083912,0.264784,0.264373,0.264081,0.264180,0.264302,0.502294,0.329670,0.711513,0.25
2,2024-06-03 13:32:00+00:00,0.267565,0.264017,0.267365,0.267592,0.002205,192.697289,0.513863,0.377955,0.559701,...,0.089591,0.264784,0.264373,0.264081,0.264180,0.264302,0.498203,0.307692,0.538036,0.75
3,2024-06-03 13:33:00+00:00,0.267347,0.265002,0.268673,0.267811,0.001637,192.801844,0.515399,0.378493,0.385417,...,0.093928,0.264784,0.264373,0.264081,0.264180,0.264302,0.493019,0.329670,0.322344,1.00
4,2024-06-03 13:34:00+00:00,0.268056,0.267740,0.269872,0.271627,0.002273,193.063506,0.528064,0.379239,0.535433,...,0.102400,0.264784,0.264373,0.264081,0.264180,0.264302,0.563066,0.714286,0.450549,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96858,2025-05-30 19:56:00+00:00,0.364390,0.362352,0.366263,0.366654,0.004605,201.763287,0.521235,0.007308,0.845570,...,0.142870,0.350011,0.351222,0.352237,0.353596,0.355277,0.756324,0.988262,0.868930,0.75
96859,2025-05-30 19:57:00+00:00,0.366463,0.363022,0.368771,0.367362,0.005688,201.875391,0.517014,0.009176,0.826211,...,0.136286,0.350675,0.351888,0.352906,0.354266,0.355949,0.758607,0.990324,0.980362,0.00
96860,2025-05-30 19:58:00+00:00,0.367172,0.363338,0.364737,0.362565,0.008021,201.745008,0.498507,0.006542,0.677570,...,0.142787,0.350750,0.352010,0.353064,0.354463,0.356192,0.677161,0.779269,0.919285,0.00
96861,2025-05-30 19:59:00+00:00,0.362317,0.357862,0.355904,0.353623,0.019922,201.065191,0.484517,0.000000,0.535055,...,0.158975,0.350750,0.352010,0.353064,0.354463,0.356192,0.557535,0.409883,0.726492,1.00


In [16]:
df_train = df[df['date'] <= train_last_date]
df_test = df[df['date'] > train_last_date]

len(df_train), len(df_test)

(83960, 12903)

In [17]:
X_train, y_train = df_train[list(features.keys())].to_numpy(), df_train['target'].to_numpy()
X_test, y_test = df_test[list(features.keys())].to_numpy(), df_test['target'].to_numpy()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((83960, 37), (83960,), (12903, 37), (12903,))

In [18]:
y_train.mean(), y_test.mean()

(0.50241184, 0.49839184)

In [55]:
import itertools 
from sklearn.base import clone
from sklearn.metrics import accuracy_score, root_mean_squared_error
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.datasets import make_classification
import lightgbm as lgb

model = lgb.LGBMClassifier(
    n_estimators=1000, 
    learning_rate=0.001,
    max_depth=5,
    num_leaves=15, 
).fit(X_train, y_train)
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

train_accuracy = accuracy_score(y_train, train_preds)
test_accuracy = accuracy_score(y_test, test_preds)
train_accuracy, test_accuracy

c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 19278, number of negative: 20170
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003378 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9435
[LightGBM] [Info] Number of data points in the train set: 39448, number of used features: 37
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.488694 -> initscore=-0.045232
[LightGBM] [Info] Start training from score -0.045232


c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


(0.5364784019468668, 0.5039641943734016)

In [327]:
model = lgb.LGBMRegressor(
    n_estimators=1000, 
    learning_rate=0.01,
    max_depth=15,
    num_leaves=31, 
).fit(X_train, y_train)
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

train_rmse = root_mean_squared_error(y_train, train_preds)
test_rmse = root_mean_squared_error(y_test, test_preds)
baseline_rmse = root_mean_squared_error(y_test, np.zeros_like(y_test) + y_test.mean())
train_rmse, test_rmse, baseline_rmse

c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006777 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9435
[LightGBM] [Info] Number of data points in the train set: 89358, number of used features: 37
[LightGBM] [Info] Start training from score 0.474832


c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


(0.4856083482130716, 0.5027996597832732, 0.49936291575431824)

In [51]:
long_predicion_mask = pd.Series(test_preds) > pd.Series(test_preds).quantile(0.99)
short_prediction_mask = pd.Series(test_preds) < pd.Series(test_preds).quantile(0.01)
y_test[long_predicion_mask].mean(), y_test[short_prediction_mask].mean()

(0.49873832, 0.47943476)

In [52]:
test_preds.mean()

0.49935667448825066

c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


(0.5209809141361729, 0.5072463768115942)

In [123]:
test_preds.mean()

0.87630194